# FNet Model for Melody Generation
This model will be trained on melodies from midi data. 

In [1]:
from os import listdir
from random import choice

from song import Song


num_of_songs = 15

midi_dir = "../examples/ghibli_dataset"
files = listdir(midi_dir)

songs = []

while num_of_songs > 0:
    s = Song(midi_dir + "/" + choice(files))
    if s.parsed:
        songs.append(s)
        num_of_songs-= 1
        
total_notes = sum([s.num_notes for s in songs])
print(f"Total number of notes: {total_notes}")

Loaded Song: Ghibli-Spirited-Away-Always-With-Me with 1 parts and 790 notes.
Loaded Song: Tonari no Totoro - Kaze no Toori Michi with 1 parts and 681 notes.
Loaded Song: Porco Rosso - Doom - Kumo no Wana with 1 parts and 218 notes.
Loaded Song: Ghibli-Princess-Mononoke-Theme-Song-Waltz with 1 parts and 1014 notes.
Loaded Song: Spirited Away - Inochi no Namae (The Name of Life) with 1 parts and 381 notes.
Loaded Song: Spirited Away - Ryuu no Shounen - from the Official Piano Solo Album with 1 parts and 448 notes.
Loaded Song: Spirited Away - The Sixth Station (1) with 1 parts and 407 notes.
Loaded Song: Ghibli-Porco-Rosso-Bygone-Days-Waltz with 1 parts and 657 notes.
Loaded Song: Ghibli-Ponyo-on-the-Cliff-by-the-Sea-Waltz with 1 parts and 727 notes.
Loaded Song: Nisemonogatari - Tetsu Kamen OST with 1 parts and 241 notes.
Loaded Song: Princess Mononoke - Ashitaka to San (Db major) with 1 parts and 464 notes.
Loaded Song: Ghibli-Laputa-The-Girl-Who-Fell-From-The-Sky-Waltz with 1 parts an

In [ ]:
# Update
# Create scale strings
# from music21.stream import Part
# from music21.duration import Duration
# from music21.note import Note

# from instrument import Instrument

# base_scale = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

# scales = []

# base_part = Part()
# for i, n in enumerate(base_scale):
#     # Limitation: each scale will have the same random attributes as the base
#     note = Note(n)
#     note.quarterLength = choice([1/2, 1/4, 1/8])
#     note.volume.velocity = choice(range(60,100))
#     note.offset = i
#     base_part.append(note)

# for i in range(12):
#     base_part = base_part.transpose(i)
#     scales.append(Instrument(base_part))
# print(len(scales))
# print(scales[0])

In [2]:
from random import randint
# Dataset augmentation
input_seq = []
output_seq = []

MAX_IN_MEASURES = 2
MAX_OUT_MEASURES = 2
STEP = 1

for song in songs:
    for part in song.parts:
        i = 0
        while i < len(part.measures):
            in_measures = randint(1, MAX_IN_MEASURES)
            out_measures = randint(1, MAX_OUT_MEASURES)
            if i + in_measures + out_measures < len(part.measures):
                in_seq = []
                for j in range(in_measures):
                    in_seq.append(str(part.measures[i + j]))
                
                out_seq = []
                for j in range(out_measures):
                    out_seq.append(str(part.measures[i + in_measures + j]))
                input_seq.append(" ".join(in_seq))
                output_seq.append(" ".join(out_seq))
            i += STEP

print(input_seq[0])
print(output_seq[0])
print(len(input_seq))
print(len(output_seq))

4/4 55 2 75 2 75 47 615 2 75 1 15 48 715 0 1 2 225 51 54 0 1 2 225 39 24 0 1 2 225 38 55 2 125 2 275 45 25 1 5 1 35 53 4/4 25 2 75 0 0 53 54 0 1 0 0 39 24 0 1 0 0 38 55 0 1 0 0 45 715 1 5 1 15 44 524 0 1 2 225 41 24 0 1 2 225 43 615 1 15 2 225 47 25 1 5 1 35 52
4/4 524 0 1 0 0 41 24 0 1 0 0 43 25 3 167 0 0 52 615 2 25 0 1 55 314 0 2 0 2 31 714 0 1 0 2 46 55 0 1 0 2 50 54 2 175 2 225 35 315 2 25 2 275 46 315 1 5 1 35 54 4/4 314 2 75 0 0 31 315 3 167 0 0 54 54 2 75 0 0 35 525 1 5 2 75 53 55 1 25 1 15 49 614 2 175 2 225 36 524 0 1 2 225 35 713 0 1 2 225 30 24 2 175 2 225 44
1512
1512


In [ ]:
# Generate vocab
# vocab = []

# for i in range(16):
#     for j in range(16):  
#         vocab.append(f"{i+1}/{j+1}")
# for i in range(10000):
#     vocab.append(str(i))

# vocab = " ".join(vocab)


In [3]:
import torch

from transformers import FNetForNextSentencePrediction, FNetTokenizerFast, FNetConfig

# Tokenize the prompts and responses
tokenizer = FNetTokenizerFast.from_pretrained("google/fnet-base")
encoded_input = tokenizer(input_seq, add_special_tokens=True, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
encoded_output = tokenizer(output_seq, add_special_tokens=True, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

/Users/jonathan/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
config = FNetConfig(
    vocab_size=9999, # the number of unique tokens in your dataset
)
# Create an instance of the model
model = FNetForNextSentencePrediction(config)

In [ ]:
from transformers import Trainer, TrainingArguments 

# Training
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='steps',     # evaluation strategy
    evaluation_steps=100,            # number of steps between evaluations
    per_device_train_batch_size=32,  # batch size
    per_device_eval_batch_size=32,   # batch size for evaluation
    weight_decay=0.01,               # weight decay
    learning_rate=5e-5,              # learning rate
    num_train_epochs=5,              # number of training epochs
    logging_dir='./logs',            # directory to save logs
    logging_steps=100                # number of steps between logging events
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    optimizers=(optimizer,),
    loss_fn=loss_fn
)

# Start training
trainer.train()